In [1]:
# -*- coding: utf-8 -*-
"""
单文件预测脚本：
1. 加载已训练好的 encoder 和 LightGBM
2. 读取单个 CSV 文件
3. 抽取“标题” → embedding → LightGBM 预测
4. 在原文件中新增“基础分数”列并保存
"""

import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from lightgbm import Booster
from tqdm import tqdm

# =============== 配置 ===============
MODEL_NAME = "shibing624/text2vec-base-chinese-sentence"
ENCODER_PATH = "../../../res/training_model/multitask_encoder.pt"
LGBM_PATH = "../../../res/training_model/multitask_lightgbm.txt"

INPUT_FILE = "../../../mid_result/training_data/merged_2w_scored_titles_LLM.csv"  # 🔹 指定要预测的文件
OUTPUT_FILE = "multitask_sentiment_title_predictions.csv"  # 🔹 输出路径

BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =============== 数据集 ===============
class TitleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=32):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["text"] = self.texts[idx]
        return item

# =============== 模型结构 ===============
import torch.nn as nn
from transformers import AutoModel

class MultiTaskModel(nn.Module):
    def __init__(self, model_name, hidden_size=768):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.head_word = nn.Linear(hidden_size, 1)
        self.head_title = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask, task_type="title"):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = out.pooler_output if out.pooler_output is not None else out.last_hidden_state[:, 0, :]
        pooled = self.dropout(pooled)
        return self.head_title(pooled)

    def get_embedding(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled = out.pooler_output if out.pooler_output is not None else out.last_hidden_state[:, 0, :]
        return pooled.detach()



# =============== 工具函数 ===============
def get_embeddings(model, dataset, batch_size=64):
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    all_vecs = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Encoding embeddings"):
            input_ids = batch["input_ids"].to(device)
            mask = batch["attention_mask"].to(device)
            vecs = model.get_embedding(input_ids, mask)
            all_vecs.append(vecs.cpu().numpy())
    return np.vstack(all_vecs)


In [2]:

# =============== 主程序 ===============
if __name__ == "__main__":
    # =============== 加载模型 ===============
    print(f"🔹 当前预测设备: {device}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    encoder = MultiTaskModel(MODEL_NAME).to(device)
    encoder.load_state_dict(torch.load(ENCODER_PATH, map_location=device))
    encoder.eval()

    lgbm_model = Booster(model_file=LGBM_PATH)

    
    print(f"📂 正在处理文件: {INPUT_FILE}")
    df = pd.read_csv(INPUT_FILE)

    if "标题" not in df.columns:
        raise ValueError(f"❌ 未找到列 '标题'，请检查文件列名。当前列为: {df.columns.tolist()}")

    texts = df["标题"].astype(str).tolist()
    dataset = TitleDataset(texts, tokenizer, max_len=32)
    X = get_embeddings(encoder, dataset, batch_size=BATCH_SIZE)
    y_pred = lgbm_model.predict(X)

    df["基础分数"] = y_pred


🔹 当前预测设备: cpu


C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_13900\1008993073.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder.load_state_dict(torch.load(ENCODER_PATH, map_locati

📂 正在处理文件: ../../../mid_result/training_data/merged_2w_scored_titles_LLM.csv


Encoding embeddings: 100%|██████████| 310/310 [08:12<00:00,  1.59s/it]


In [3]:

    df.to_csv(OUTPUT_FILE, index=False, encoding="utf-8-sig")
    print(f"✅ 预测完成，结果已保存至: {OUTPUT_FILE}")

✅ 预测完成，结果已保存至: multitask_sentiment_title_predictions.csv
